In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

### Import mamtorch library
To install mamtorch, in mamtorch root folder, "pip install ." (to improve compiling time, do this after installing ninja through "pip install ninja")

In [2]:
import mamtorch as mam

### Select the GPU
Currently, MAM kernels are implemented only for usage on GPU

In [3]:
# Select GPU
gpu_id = 6
# Check if the GPU is available, and if so, use it
device = torch.device(f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu")
# You need a gpu to use MAM kernel! (No cpu-based implementation available)
if(device == "cpu"):
    raise "No GPU device available! MAM kernels are not available."

### Define a simple feedforward DNN containing a MAM layer

In [ ]:
compute_exact = False
vcon_epochs = 0
splits = 1

# Define a simple feedforward neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes, vcon_steps):
        super(SimpleNN, self).__init__()
        # Instantiate a MAC fc layer
        self.fc1 = nn.Linear(input_size, hidden_size1)
        #self.fc1 = mam.nn.FullyConnected(input_size, hidden_size1, bias=True, splits=splits, vcon_steps=vcon_steps, vcon_type='exponential', compute_exact=compute_exact)
        self.relu1 = nn.ReLU()
        
        # Instantiate a MAM fc layer
        #self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc2 = mam.nn.FullyConnected(hidden_size1, hidden_size2, bias=True, splits=splits, vcon_steps=vcon_steps, vcon_type='exponential', compute_exact=compute_exact)
        self.relu2 = nn.ReLU()
        
        # Instantiate the output layer
        self.fc3 = nn.Linear(hidden_size2, num_classes)
        #self.fc3 = mam.nn.FullyConnected(hidden_size2, num_classes, bias=True, splits=splits, vcon_steps=vcon_steps, vcon_type='exponential', compute_exact=compute_exact)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        print(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x
    
# Hyperparameters
input_size = 28 * 28  # MNIST image size
hidden_size1 = 512
hidden_size2 = 256
num_classes = 10
learning_rate = 0.001
batch_size = 128
num_epochs = 100

Load MNIST dataset and apply transformations

In [5]:
num_workers = 2 # increase this to use more threads to manage data

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valtest_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
val_dataset, test_dataset = torch.utils.data.random_split(valtest_dataset, [0.5, 0.5], generator=torch.Generator().manual_seed(42))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

Initialize the model, loss function, and optimizer

In [6]:
vcon_steps = vcon_epochs*len(train_loader)
model = SimpleNN(input_size, hidden_size1, hidden_size2, num_classes, vcon_steps).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train the network

In [7]:
# Initialize the selection matrix list.
# Here, for each training epoch, we store the number of times each interconnection has been used
# I.E., the selection count
selection_matrix_list = []

# Training loop
for epoch in range(num_epochs):
    start_time = time.perf_counter()
    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    model.train() # se training mode

    correct = 0
    total = 0
    running_loss = 0
    total_step = len(train_loader)

    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28 * 28).to(device)  # Flatten the input images
        outputs = model(images)
        
        loss = criterion(outputs, labels.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss

        # get the predicted class
        _, predicted = torch.max(outputs.data, 1)

        # evaluate the correct values agaist the total evaluated
        correct += (predicted == labels.to(device)).sum().item()
        total += labels.size(0)

        # update beta values
        #model.fc1.vcon_step()
        model.fc2.vcon_step()
        #model.fc3.vcon_step()

        print(f'Training [{i + 1}/{total_step}], Loss: {running_loss/total:.3e}, Acc: {correct/total*100:.1f}%', end='\r')
    print(f'Training [{total_step}/{total_step}], Loss: {running_loss/total:.3e}, Acc: {correct/total*100:.1f}%')

    model.eval() # set evaluation mode
    
    correct = 0
    total = 0
    running_loss = 0
    total_step = len(val_loader)
    for images, labels in val_loader:
        images = images.view(-1, 28 * 28).to(device)  # Flatten the input images
        outputs = model(images)
        loss = criterion(outputs, labels.to(device))
        
        running_loss += loss
        
        # get the predicted class
        _, predicted = torch.max(outputs.data, 1)
        
        # evaluate the correct values agaist the total evaluated
        correct += (predicted == labels.to(device)).sum().item()
        total += labels.size(0)

        print(f'Validation [{i + 1}/{total_step}], Loss: {running_loss/total:.3e}, Acc: {correct/total*100:.1f}%', end='\r')
    print(f'Validation [{total_step}/{total_step}], Loss: {running_loss/total:.3e}, Acc: {correct/total*100:.1f}%')
    
    # update the value of beta for vanishing contributes
    print("Latest beta value:", model.fc2.beta)
    
    print(f"Elapsed time = {time.perf_counter()-start_time:.3f} s")
    
print("Training end.")

Epoch [1/100]
tensor([[-0.0197, -0.0378,  0.0138,  ..., -0.0274,  0.0238,  0.0172],
        [-0.0434, -0.0064,  0.0233,  ..., -0.0447,  0.0305,  0.0428],
        [-0.0265, -0.0353,  0.0047,  ..., -0.0438,  0.0272,  0.0180],
        ...,
        [-0.0211, -0.0051,  0.0143,  ..., -0.0242,  0.0363,  0.0258],
        [-0.0249, -0.0005, -0.0095,  ..., -0.0234,  0.0201,  0.0367],
        [-0.0140, -0.0257, -0.0090,  ..., -0.0444,  0.0175,  0.0380]],
       device='cuda:6', grad_fn=<ViewBackward0>)
tensor([[-0.0354, -0.0246,  0.0268,  ..., -0.0248,  0.0481,  0.0392],
        [-0.0407, -0.0271, -0.0105,  ..., -0.0658,  0.0114,  0.0341],
        [-0.0284, -0.0170,  0.0141,  ..., -0.0379,  0.0120,  0.0316],
        ...,
        [-0.0350, -0.0103, -0.0011,  ..., -0.0532,  0.0216,  0.0426],
        [-0.0197, -0.0132,  0.0070,  ..., -0.0353,  0.0056,  0.0242],
        [-0.0180, -0.0136, -0.0028,  ..., -0.0521,  0.0131,  0.0395]],
       device='cuda:6', grad_fn=<ViewBackward0>)
tensor([[-0.0196, -0

KeyboardInterrupt: 

In [ ]:
# Test the model on the test dataset
model.eval() # set evaluation mode

correct = 0
total = 0
running_loss = 0
total_step = len(val_loader)
for images, labels in test_loader:
    images = images.view(-1, 28 * 28).to(device)  # Flatten the input images
    outputs = model(images)
    loss = criterion(outputs, labels.to(device))
    
    running_loss += loss
    
    # get the predicted class
    _, predicted = torch.max(outputs.data, 1)
    
    # evaluate the correct values agaist the total evaluated
    correct += (predicted == labels.to(device)).sum().item()
    total += labels.size(0)

    print(f'Test [{i + 1}/{total_step}], Loss: {running_loss/total:.3e}, Acc: {correct/total*100:.1f}%', end='\r')
print(f'Test [{total_step}/{total_step}], Loss: {running_loss/total:.3e}, Acc: {correct/total*100:.1f}%')

Test [79/79], Loss: 7.321e-03, Acc: 82.9%%
